<a href="https://colab.research.google.com/github/Whereamiactually/lyceumcompling11/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинный перевод
*(частично материалы курса ФиКЛ, спасибо авторам и Татьяне Казаковой)*

### Правила
Традиционно: можно долго прописывать правила (соответствия слов, трансформацию синтаксиса и т.д.)

### Статистический машинный перевод

Наиболее частый - PBMT - phrase-based machine translation. Есть пословный, синтаксический и другие. В парах предложений в большом корпусе статистически определяются соответствия кусочков предложений (фраз) и эта информация используется для перевода нового текста. Часто используется как базовая модель, если нет достаточного корпуса для нейронного.

### NMT - neural machine translation

Обучается нейронная сеть на большом корпусе, которая учится предсказывать наилучший перевод.

Если не правила, нам нужны параллельные корпуса с выравниванием по предложению, например, [OPUS](https://opus.nlpl.eu/).



### Как оценивается качество?

Например, с помощью BLEU (bilingual evaluation understudy) - доля слов или чаще n-грамм, которые совпали в варианте перевода и ответами (0 - все плохо, 1 - отлично, но так не бывает даже у человека). Считается, что 4-граммы наиболее хорошо описывают хорошую человеческую речь.

### Попробуем пословное выравнивание

Будет использовать программу fast_align.

In [1]:
! git clone https://github.com/clab/fast_align.git

Cloning into 'fast_align'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 213 (delta 2), reused 4 (delta 2), pack-reused 204
Receiving objects: 100% (213/213), 70.68 KiB | 540.00 KiB/s, done.
Resolving deltas: 100% (110/110), done.


In [2]:
! mkdir /content/fast_align/build # создаем папку для сборки

In [3]:
! pwd
# переходим в папку
%cd fast_align/build
! pwd

/content
/content/fast_align/build
/content/fast_align/build


In [4]:
! cmake .. # собираем

CMake Warning (dev) at CMakeLists.txt:1 (project):
  cmake_minimum_required() should be called prior to this top-level project()
  call.  Please see the cmake-commands(7) manual for usage documentation of
  both commands.
This warning is for project developers.  Use -Wno-dev to suppress it.

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Deprecation Warning at CMakeLists.txt:2 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or u

In [5]:
! make # собираем

[ 16%] Building CXX object CMakeFiles/fast_align.dir/src/fast_align.cc.o
[ 33%] Building CXX object CMakeFiles/fast_align.dir/src/ttables.cc.o
[ 50%] Linking CXX executable fast_align
[ 50%] Built target fast_align
[ 66%] Building CXX object CMakeFiles/atools.dir/src/alignment_io.cc.o
[ 83%] Building CXX object CMakeFiles/atools.dir/src/atools.cc.o
[100%] Linking CXX executable atools
[100%] Built target atools


In [6]:
# возвращаемся в домашнюю папку
%cd ..
%cd ..

/content/fast_align
/content


Скачиваем с OPUS небольшой корпус книг.

In [7]:
! wget https://object.pouta.csc.fi/OPUS-Books/v1/moses/en-ru.txt.zip

--2023-10-20 08:38:32--  https://object.pouta.csc.fi/OPUS-Books/v1/moses/en-ru.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1613419 (1.5M) [application/zip]
Saving to: ‘en-ru.txt.zip’

en-ru.txt.zip       100%[===================>]   1.54M   901KB/s    in 1.7s    

2023-10-20 08:38:36 (901 KB/s) - ‘en-ru.txt.zip’ saved [1613419/1613419]



In [8]:
! unzip en-ru.txt.zip

Archive:  en-ru.txt.zip
  inflating: Books.en-ru.en          
  inflating: Books.en-ru.ru          
  inflating: Books.en-ru.ids         
  inflating: README                  


Преобразуем данные в тот вид, в котором они принимаются программой.

In [9]:
! paste Books.en-ru.en Books.en-ru.ru -d "\t" | sed 's/\t/ ||| /' > Books.en-ru

In [10]:
! head Books.en-ru

Anna Karenina ||| Анна Каренина
Leo Tolstoy ||| Толстой Лев Николаевич
Vengeance is mine; I will repay. ||| Мне отмщение, и аз воздам
VOLUME ONE PART I ||| ЧАСТЬ ПЕРВАЯ
CHAPTER I ||| I
ALL HAPPY FAMILIES resemble one another, but each unhappy family is unhappy in its own way. ||| Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.
Everything was upset in the Oblonskys' house. ||| Все смешалось в доме Облонских.
The wife had discovered an intrigue between her husband and their former French governess, and declared that she would not continue to live under the same roof with him. ||| Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме.
This state of things had now lasted for three days, and not only the husband and wife but the rest of the family and the whole household suffered from it. ||| Положение это продолжалось уже третий день и мучительно чувствовалось и самими

In [11]:
! mv ./fast_align ./fast_align_git # переименуем папку, а то она будет с таким же именем

In [12]:
! cp ./fast_align_git/build/fast_align ./fast_align
! cp ./fast_align_git/build/atools ./atools

In [13]:
! ./fast_align -i Books.en-ru -d -o -v > forward.align

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................
expected target length = source length * 0.86155
ITERATION 1
.................
  log_e likelihood: -5.8083e+06
  log_2 likelihood: -8.3796e+06
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.174998
       size counts: 1614
ITERATION 2
.................
  log_e likelihood: -2.28139e+06
  log_2 likelihood: -3.29134e+06
     cross entropy: 11.7431
        perplexity: 3427.88
      posterior p0: 0.0906316
 posterior al-feat: -0.117567
       size counts: 1614
  1  model al-feat: -0.209743 (tension=4)
  2  model al-feat: -0.164009 (tension=5.84353)
  3  model al-feat: -0.147483 (tension=6.77237)
  4  model al-feat: -0.138632 (tension=7.37069)
  5  model al-feat: -0.133146 (tension=7.79201)
  6  model al-feat: -0.12946 (tension=8.1036)
  7  model al-feat: -0.126852 (tension=8.34148)
  8  model al-feat: -0.124936 (tension=8.52718)
     final tension: 8.67458
ITERATION 3
..............

In [14]:
! ./fast_align -i Books.en-ru -d -o -v -r > reverse.align

ARG=i
ARG=d
ARG=o
ARG=v
ARG=r
INITIAL PASS 
.................
expected target length = source length * 1.26205
ITERATION 1
.................
  log_e likelihood: -7.05339e+06
  log_2 likelihood: -1.01759e+07
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.172934
       size counts: 1614
ITERATION 2
.................
  log_e likelihood: -2.29682e+06
  log_2 likelihood: -3.31361e+06
     cross entropy: 9.73557
        perplexity: 852.506
      posterior p0: 0.104449
 posterior al-feat: -0.121262
       size counts: 1614
  1  model al-feat: -0.138919 (tension=4)
  2  model al-feat: -0.132396 (tension=4.35315)
  3  model al-feat: -0.128507 (tension=4.57582)
  4  model al-feat: -0.126068 (tension=4.72073)
  5  model al-feat: -0.124489 (tension=4.81684)
  6  model al-feat: -0.123445 (tension=4.88138)
  7  model al-feat: -0.122747 (tension=4.92505)
  8  model al-feat: -0.122276 (tension=4.95476)
     final tension: 4.97504
ITERATION 3
.....

In [15]:
! ./atools -i forward.align -j reverse.align -c grow-diag-final-and > result.align

In [16]:
! head result.align

0-0 1-1
0-0 0-1 1-2
0-0 1-1 2-1 2-2 3-3 4-3 5-4
0-0 1-0 2-0 2-1 3-1
0-0 1-0
0-1 1-0 2-1 2-2 3-3 4-3 5-4 7-4 7-5 8-6 9-9 11-7 11-8 14-11 15-11
0-0 1-1 2-1 3-2 4-3 5-4 6-4
1-0 2-1 2-2 3-2 4-4 4-5 5-5 6-6 7-3 8-3 9-7 10-10 11-8 11-9 13-12 14-11 14-13 15-12 16-16 17-14 18-16 19-17 20-17 21-18 22-19 23-22 24-22 25-23 26-24 27-20 28-21
1-0 3-1 4-3 5-3 6-2 7-2 8-4 9-6 10-6 10-7 13-10 14-11 15-8 15-9 16-9 18-10 19-11 20-12 21-14 22-15 23-12 23-13 23-16 24-14 27-17 28-16 29-17
0-0 1-0 2-5 3-6 4-7 5-6 6-7 8-9 9-10 10-9 11-11 12-12 13-13 15-9 16-8 17-20 18-14 21-22 22-21 24-15 24-16 25-24 26-29 29-23 30-21 31-28 33-25 34-30 34-31


In [ ]:
# а так было
! head Books.en-ru

Anna Karenina ||| Анна Каренина
Leo Tolstoy ||| Толстой Лев Николаевич
Vengeance is mine; I will repay. ||| Мне отмщение, и аз воздам
VOLUME ONE PART I ||| ЧАСТЬ ПЕРВАЯ
CHAPTER I ||| I
ALL HAPPY FAMILIES resemble one another, but each unhappy family is unhappy in its own way. ||| Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.
Everything was upset in the Oblonskys' house. ||| Все смешалось в доме Облонских.
The wife had discovered an intrigue between her husband and their former French governess, and declared that she would not continue to live under the same roof with him. ||| Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме.
This state of things had now lasted for three days, and not only the husband and wife but the rest of the family and the whole household suffered from it. ||| Положение это продолжалось уже третий день и мучительно чувствовалось и самими

Можно обучить свою нейросеть. Вот тут можно посмотреть на [код-пример](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/machine_translation/MT_transformer_tf.ipynb).

### И, конечно, готовое решение

Их очень много, но для популярных языков.

In [17]:
! pip3 install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=80f7850255c08b87dc28f8c64699931b428099c4096b910a5f1a5d7450310b45
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [18]:
from googletrans import Translator, constants
from pprint import pprint

In [19]:
translator = Translator()

In [23]:
translation = translator.translate("Mi casa es su casa")
print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

Mi casa es su casa (es) --> My house is your house (en)


In [25]:
translation = translator.translate("Вы умеете читать по-арабски?",
                                   src = "ru", dest = "ar")
print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

Вы умеете читать по-арабски? (ru) --> هل تستطيع قراءة اللغة العربية؟ (ar)


In [ ]:
pprint(translation.extra_data)

Также можно переводить набор предложений.

In [30]:
translations = translator.translate(["Я обожаю ромашки.", "Каждый раз, когда я выхожу в поле, я их собираю.", "Хотите, подарю вам букетик?", "Иначе я подарю его этой девочке. Она их любит."], src = "ru", dest = "de")
for translation in translations:
    print("Translation > ", translation.text)

Translation >  Ich liebe Gänseblümchen.
Translation >  Jedes Mal, wenn ich aufs Feld gehe, sammle ich sie ein.
Translation >  Möchten Sie, dass ich Ihnen einen Blumenstrauß schenke?
Translation >  Sonst gebe ich es diesem Mädchen. Sie liebt sie.


Мы можем также просто попросить определить язык.

In [28]:
result = translator.detect("Welcome to our tutorial!")
print(result)

Detected(lang=en, confidence=1)


In [32]:
result = translator.detect("Kuinka monta karhua sinulla on?")
print(result)

Detected(lang=fi, confidence=1)


In [33]:
translation = translator.translate("Kuinka monta karhua sinulla on?")
print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

Kuinka monta karhua sinulla on? (fi) --> How many bears do you have? (en)


Мы можем попробовать совместить распознавание текста с изображения и перевод.

In [34]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 5s (1,008 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [35]:
!pip install pytesseract

In [36]:
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [39]:
import cv2
from google.colab import files

Распознаем текст с картинки.

In [40]:
loaded_file = files.upload() # просим пользователя дать картинку

Saving Rus text_page-0001.jpg to Rus text_page-0001.jpg


In [42]:
!sudo apt install tesseract-ocr-rus
# либо можно просто написать !sudo apt install tesseract-ocr-all, чтобы установить все языки
# это удобно, если вы работает не в google.colab...

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-rus is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [43]:
filename = list(loaded_file.keys())[0]
img = cv2.imread(filename) # считываем картинку
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # делаем её нужной цветовой гаммы
print(pytesseract.image_to_osd(img)) # проверяем ориентацию и вообще есть ли распознаваемый текст на картинке

Page number: 0
Orientation in degrees: 0
Rotate: 0
Orientation confidence: 6.64
Script: Cyrillic
Script confidence: 3.75



In [44]:
config = r'--oem 3 --psm 6'
text_image = pytesseract.image_to_string(img, config = config, lang = 'rus')
print(text_image)

СИНТАКСИС
° Предмет синтаксиса.

Синтаксис - это раздел грамматики, исслодующий правила соче-
тания слов между собой ‚обобщенные значения этих сочетаний, а так-
зе типы предложений по их структуре и цели высказывания.В процес-
се речи слова связываются друг с другом по определенным правилам.
К этим правилам относятся как типы связи слов(проявляющиеся в от-
дельных случаях в словоизменительных морфемах), так и порядок их
расположения. При монологе речь распадается (разделяется) на от-
резки,которые, с одной стороны, обусловлены особенностями функци-
онирования органов речи(обычно речь человека ведетоя во время вы-
доха, хотя может произноситься и при вдохе),с другой - зависят от
смыслового членения.Как правило, членение речи, вызываемое пауза-
ми, обусловленное особенностями функционирования органов речи, со-
зпадавт оо смысловым, но может и не совпадать.

Переходы между паузами, совпадающие со смысловым членением, -
назовем их синтагмами, - в свою очередь, разделяются на мелкие от-

- 

Делим на предложения, чтобы потом исправлять и переводить каждое по отдельности. Получится небольшой параллельный "корпус".  

In [45]:
pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=b6bffea55228d9779597883c323f306379943928435debdb3e98c7d79facb618
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=b42bbb92c4689b2ddc166dcf8ac7d1679caff834b88d85726aad324056455367
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [87]:
from razdel import sentenize
from razdel import tokenize

In [88]:
sentences = list(sentenize(text_image))
pprint(sentences)

[Substring(0, 31, 'СИНТАКСИС\n° Предмет синтаксиса.'),
 Substring(33, 292, 'Синтаксис - это раздел грамматики, исслодующий правила соче-\nтания слов между собой ‚обобщенные значения этих сочетаний, а так-\nзе типы предложений по их структуре и цели высказывания.В процес-\nсе речи слова связываются друг с другом по определенным правилам.'),
 Substring(293, 437, 'К этим правилам относятся как типы связи слов(проявляющиеся в от-\nдельных случаях в словоизменительных морфемах), так и порядок их\nрасположения.'),
 Substring(438, 866, 'При монологе речь распадается (разделяется) на от-\nрезки,которые, с одной стороны, обусловлены особенностями функци-\nонирования органов речи(обычно речь человека ведетоя во время вы-\nдоха, хотя может произноситься и при вдохе),с другой - зависят от\nсмыслового членения.Как правило, членение речи, вызываемое пауза-\nми, обусловленное особенностями функционирования органов речи, со-\nзпадавт оо смысловым, но может и не совпадать.'),
 Substring(868, 1060, 'Пер

In [89]:
import re
new_sentences = []
for start, end, sentence in list(sentences):
  new_sentence = re.sub('\n|-\n', '', sentence)
  new_sentences.append(new_sentence)
print(new_sentences)

['СИНТАКСИС° Предмет синтаксиса.', 'Синтаксис - это раздел грамматики, исслодующий правила сочетания слов между собой ‚обобщенные значения этих сочетаний, а такзе типы предложений по их структуре и цели высказывания.В процессе речи слова связываются друг с другом по определенным правилам.', 'К этим правилам относятся как типы связи слов(проявляющиеся в отдельных случаях в словоизменительных морфемах), так и порядок ихрасположения.', 'При монологе речь распадается (разделяется) на отрезки,которые, с одной стороны, обусловлены особенностями функционирования органов речи(обычно речь человека ведетоя во время выдоха, хотя может произноситься и при вдохе),с другой - зависят отсмыслового членения.Как правило, членение речи, вызываемое паузами, обусловленное особенностями функционирования органов речи, созпадавт оо смысловым, но может и не совпадать.', 'Переходы между паузами, совпадающие со смысловым членением, назовем их синтагмами, - в свою очередь, разделяются на мелкие от- резки, характе

Переводим.

In [92]:
translations = translator.translate(new_sentences, src = "ru", dest = "de")
for translation in translations:
    print("Translation > ", translation.text)

Translation >  SYNTAX° Gegenstand der Syntax.
Translation >  Syntax ist ein Abschnitt der Grammatik, der die Regeln für die Kombination von Wörtern untereinander, die verallgemeinerten Bedeutungen dieser Kombinationen sowie die Arten von Sätzen entsprechend ihrer Struktur und dem Zweck der Aussage untersucht. Im Sprechprozess sind Wörter nach bestimmten Regeln miteinander verbunden sind.
Translation >  Diese Regeln umfassen sowohl die Art der Verbindungen zwischen Wörtern (die sich in einigen Fällen in Flexionsmorphemen manifestieren) als auch die Reihenfolge ihrer Anordnung.
Translation >  Während eines Monologs zerfällt (teilt) die Sprache in Segmente, die einerseits durch die Besonderheiten der Funktionsweise der Sprechorgane bestimmt werden (normalerweise spricht eine Person beim Ausatmen, kann aber auch beim Einatmen ausgesprochen werden), andererseits hängen sie von der semantischen Teilung ab. In der Regel ist die durch Pausen verursachte Teilung der Sprache aufgrund der Besonde

Можно написать несложный код, который соотносит каждое предложение с его переводом.